# 🤖 LMFast: Building AI Agents

**Create intelligent tool-using agents with Small Language Models!**

## What You'll Learn
- Create agents that use tools
- Implement ReAct (Reason + Act) loops
- Build specialized agents (Code, Data)
- Advanced reasoning with ThinkingAgent

## Why SLM Agents?
- Run locally, no API costs
- Privacy-preserving
- Fast response times
- Customizable behavior

**Time to complete:** ~15 minutes

## 1️⃣ Setup

In [ ]:
!pip install -q lmfast[all]

import lmfast
lmfast.setup_colab_env()

import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2️⃣ Load a Base Model for the Agent

In [ ]:
from lmfast.inference import SLMServer

# Load a capable instruction-tuned model
# Using SmolLM for speed, but Qwen2.5-0.5B is better for tools
model = SLMServer("HuggingFaceTB/SmolLM-360M-Instruct")

# Create a generate function for the agent
def generate(prompt: str) -> str:
    return model.generate(
        prompt, 
        max_new_tokens=256,
        temperature=0.7
    )

# Test
print(generate("What is 2 + 2?"))

## 3️⃣ Define Tools

Tools are just Python functions that the agent can call.

In [ ]:
import math
from datetime import datetime

def calculate(expression: str) -> str:
    """Evaluate a mathematical expression. Example: calculate('2 + 2')"""
    try:
        # Safe evaluation
        allowed_names = {
            'abs': abs, 'round': round, 'min': min, 'max': max,
            'sum': sum, 'pow': pow, 'sqrt': math.sqrt,
            'sin': math.sin, 'cos': math.cos, 'tan': math.tan,
            'log': math.log, 'exp': math.exp, 'pi': math.pi
        }
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return str(result)
    except Exception as e:
        return f"Error: {e}"

def get_current_time() -> str:
    """Get the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def search_web(query: str) -> str:
    """Search the web for information. Returns a summary."""
    # Simulated search results
    results = {
        "weather": "Current weather: Sunny, 25°C",
        "news": "Top news: Technology advances in AI continue",
        "python": "Python is a popular programming language created by Guido van Rossum",
    }
    for key, value in results.items():
        if key in query.lower():
            return value
    return f"Search results for '{query}': No specific results found."

# Test tools
print(f"calculate('sqrt(16)'): {calculate('sqrt(16)')}")
print(f"get_current_time(): {get_current_time()}")
print(f"search_web('python'): {search_web('python')}")

## 4️⃣ Create a Basic Agent

In [ ]:
from lmfast.agents.core import Agent, Tool

# Create agent with tools
agent = Agent(
    model_generate_fn=generate,
    tools=[calculate, get_current_time, search_web],
    system_prompt="""You are a helpful AI assistant with access to tools.
When you need to use a tool, output JSON like: {"tool": "tool_name", "args": {"arg": "value"}}
Always try to use tools to provide accurate information."""
)

print("Agent created with tools:")
for name, tool in agent.tools.items():
    print(f"  - {name}: {tool.description[:60]}...")

## 5️⃣ Run the Agent

In [ ]:
# Test queries
queries = [
    "What is the square root of 144?",
    "What time is it right now?",
    "Search for information about Python programming.",
    "Calculate 15 * 7 + 23"
]

print("🤖 Agent Responses")
print("=" * 50)

for query in queries:
    print(f"\n📝 User: {query}")
    response = agent.run(query)
    print(f"🤖 Agent: {response}")
    print("-" * 40)

## 6️⃣ Advanced: ThinkingAgent (Chain-of-Thought)

For complex reasoning tasks, use the ThinkingAgent with test-time compute scaling.

In [ ]:
from lmfast.reasoning import ThinkingAgent, reason

# Create a thinking agent
thinker = ThinkingAgent(
    model_generate_fn=generate,
    n=3  # Generate 3 candidate solutions
)

# Test with a reasoning problem
problems = [
    "If a train travels 60 km/h for 2 hours, then 80 km/h for 1.5 hours, what's the total distance?",
    "A bat and ball cost $1.10 together. The bat costs $1 more than the ball. How much does the ball cost?",
    "If 5 machines make 5 widgets in 5 minutes, how many minutes for 100 machines to make 100 widgets?"
]

print("🧠 ThinkingAgent - Best-of-N Reasoning")
print("=" * 50)

for problem in problems:
    print(f"\n📝 Problem: {problem}")
    
    # Best-of-N sampling
    answer = thinker.reason(problem, method="best_of_n")
    print(f"🧠 Answer: {answer[:300]}...")
    print("-" * 40)

## 7️⃣ One-Line Reasoning

In [ ]:
# Simple functional API
from lmfast import reason

answer = reason(
    model_fn=generate,
    problem="What is 15% of 80?",
    method="cot",  # Chain of Thought
    n=3
)

print(f"Answer: {answer}")

## 8️⃣ Specialized Agents: CodeAgent

In [ ]:
from lmfast.agents.specialized import CodeAgent

# Create code agent
code_agent = CodeAgent(model_generate_fn=generate)

# Test code generation and execution
code_tasks = [
    "Write a function to calculate fibonacci numbers",
    "Create a simple function that reverses a string",
]

print("💻 CodeAgent")
print("=" * 50)

for task in code_tasks:
    print(f"\n📝 Task: {task}")
    result = code_agent.run(task)
    print(f"💻 Result: {result[:400]}...")
    print("-" * 40)

## 9️⃣ Build a Custom Multi-Tool Agent

In [ ]:
# Define custom tools for a specific use case

def get_stock_price(symbol: str) -> str:
    """Get the current stock price for a given symbol."""
    # Simulated prices
    prices = {
        "AAPL": 185.42,
        "GOOGL": 175.23,
        "MSFT": 420.15,
        "NVDA": 875.50
    }
    symbol = symbol.upper()
    if symbol in prices:
        return f"{symbol}: ${prices[symbol]}"
    return f"Stock {symbol} not found"

def convert_currency(amount: float, from_curr: str, to_curr: str) -> str:
    """Convert between currencies. Supports USD, EUR, GBP, JPY."""
    rates = {
        "USD": 1.0,
        "EUR": 0.92,
        "GBP": 0.79,
        "JPY": 149.50
    }
    try:
        usd_amount = float(amount) / rates.get(from_curr.upper(), 1.0)
        converted = usd_amount * rates.get(to_curr.upper(), 1.0)
        return f"{amount} {from_curr} = {converted:.2f} {to_curr}"
    except:
        return "Conversion error"

# Create finance agent
finance_agent = Agent(
    model_generate_fn=generate,
    tools=[get_stock_price, convert_currency, calculate],
    system_prompt="""You are a financial assistant. Use available tools to help with:
- Stock prices
- Currency conversions  
- Calculations
Always use tools when relevant."""
)

# Test
queries = [
    "What's the current price of NVDA stock?",
    "Convert 100 USD to EUR",
    "If I buy 10 shares of AAPL at current price, how much will it cost?"
]

print("💰 Finance Agent")
print("=" * 50)

for query in queries:
    print(f"\n📝 Query: {query}")
    response = finance_agent.run(query)
    print(f"💰 Response: {response}")

## 🔟 Serve Agent via MCP (Claude, Cursor integration)

In [ ]:
# Note: MCP server runs as a background process
# This is for demonstration - in practice, run from command line

from lmfast.mcp.server import LMFastMCPServer

# Create MCP server (don't run in notebook - blocks execution)
# server = LMFastMCPServer(
#     model_path="HuggingFaceTB/SmolLM-360M-Instruct",
#     name="lmfast-agent"
# )
# server.run()  # This blocks

print("To run MCP server from command line:")
print("")
print("  lmfast serve ./my_model --mcp")
print("")
print("Configure Claude Desktop (~/.config/claude/claude_desktop_config.json):")
print('''
{
  "mcpServers": {
    "lmfast": {
      "command": "lmfast",
      "args": ["serve", "./my_model", "--mcp"]
    }
  }
}
''')

## 🎉 Summary

You've learned how to:
- ✅ Create agents with custom tools
- ✅ Use ThinkingAgent for complex reasoning
- ✅ Build specialized agents (Code, Finance)
- ✅ Serve agents via MCP protocol

### Agent Design Tips

| Tip | Why |
|-----|-----|
| Clear tool descriptions | Helps model choose right tool |
| Simple return types | String outputs are easiest |
| Good system prompts | Guide agent behavior |
| Error handling in tools | Graceful failure |

### Next Steps
- `10_reasoning_agents.ipynb` - Advanced reasoning
- `11_mcp_integration.ipynb` - Full MCP setup
- `12_rag_agents.ipynb` - RAG-augmented agents